In [5]:
import pandas as pd
import numpy as np
import pickle as pkl

In [2]:
with open('dataset/data/train_set.csv') as f:
    train = f.readlines()[1:]
# test = pd.read_csv('data/test_a.csv')
with open('dataset/data/test_b.csv') as f:
    test = f.readlines()[1:]
    x_test = []
    for s in test:
        x_test.append(s.strip())

In [3]:
x, y = [], []
L = []
category = set()

for line in train:
    y_, x_ = line.split('\t')
    category.add(int(y_))
    x.append(x_.strip())
    y.append(y_.strip())
    L.append(len(x_.split()))
    
# with open('dataset/data/class.txt','w') as f:
#     for c in sorted(category):
#         f.write(str(c)+'\n')
print(len(y))
print(np.mean(L), np.max(L), np.min(L), np.std(L))

200000
907.20711 57921 2 996.0265463156331


In [30]:
def C(k):
    i=0
    for s in L:
        if s<=k:
            i+=1
    print('%d长度可以覆盖 百分之%.5f样本'%(k,i/len(L)*100))
C(500)
C(1000)
C(2000)
C(3000)
C(4000)
C(5000)

500长度可以覆盖 百分之36.24450样本
1000长度可以覆盖 百分之69.41350样本
2000长度可以覆盖 百分之92.02250样本
3000长度可以覆盖 百分之97.11500样本
4000长度可以覆盖 百分之98.79400样本
5000长度可以覆盖 百分之99.44150样本


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.2, random_state=0)

In [7]:
#train_all
with open('dataset/data/train_all.txt','w') as f:
    for i in range(len(y_train)):
        f.write(x_train[i]+'\t'+y_train[i]+'\n')
    for i in range(len(y_dev)):
        f.write(x_dev[i]+'\t'+y_dev[i]+'\n')

# train dev test
# with open('dataset/data/train.txt','w') as f:
#     for i in range(len(y_train)):
#         f.write(x_train[i]+'\t'+y_train[i]+'\n')        
# with open('dataset/data/dev.txt','w') as f:
#     for i in range(len(y_dev)):
#         f.write(x_dev[i]+'\t'+y_dev[i]+'\n')
# with open('dataset/data/test.txt','w') as f:
#     for i in range(len(test)):
#         f.write(test[i])

#corpus
# with open('dataset/data/corpus.txt','w') as f:
#     for i in range(len(y_train)):
#         f.write(x_train[i]+'\n')
#     for i in range(len(y_dev)):
#         f.write(x_dev[i]+'\n')    
#     for i in range(len(test)):
#         f.write(test[i])
with open('dataset/data/train_all.txt','r') as f:
    f = f.readlines()
print(len(f))

200000


## 预训练词向量

In [13]:
import time
import cython
import numpy as np
from gensim.models import word2vec #必须按照[[],[]]才能保留原词！
from gensim.models.word2vec import LineSentence
# from sklearn import preprocessing
# from sklearn import svm
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.decomposition import PCA
# from sklearn.externals import joblib
# from sklearn import metrics

In [22]:
%%time
w2v_size = 300
X = []
with open('dataset/data/corpus.txt', encoding='utf-8') as f:
    lines = f.readlines()[:] 
    X = [[] for i in range(len(lines))]
    i = 0
    for line in lines:
        line = line.strip()
        X[i] = line.split(' ')#由于这一整个是str，但是w2v要一个个词
        i += 1


CPU times: user 9.61 s, sys: 9.82 s, total: 19.4 s
Wall time: 20 s


In [24]:
%%time
k=None
model = word2vec.Word2Vec(X[:k], size=w2v_size, min_count=5, workers=-1, iter=10)
print(model)
# model.build_vocab(X[:k])  # prepare the model vocabulary
# model.train(X[:k], total_examples=len(X[:k]), epochs=model.iter)

model.save('dataset/data/w2v/w2v_300_mincount5.model')
    # model.wv.save_word2vec_format("444.model.bin", binary=True)
    #增量学习 incremental/online learning
    # model = word2vec.Word2Vec.load('w2v.model')
    #model = word2vec.Word2Vec.load_word2vec_format("444.model.bin", binary=True)
    #model.build_vocab(X, update=True) #其实文档里都有的，虽然没有明确标出
    #model.train(X, total_examples=len(X), epochs=model.iter)

    # print(model.wv.vocab)

Word2Vec(vocab=6151, size=300, alpha=0.025)
CPU times: user 43.3 s, sys: 9.41 s, total: 52.7 s
Wall time: 51.5 s


In [20]:
with open('dataset/data/bert_emb.npy','rb') as f:
    bert_emb = np.load(f)
with open('dataset/bert-mini/vocab.txt') as f:
    vocab_bert = {w[:-1]:i for i,w in enumerate(f.readlines())}
print(bert_emb.shape)

V = pkl.load(open('dataset/data/vocab_tra_all.pkl','rb'))
emb = np.zeros((len(V), 256)) #* 0.000962582706903456
for w in V.keys():
    if w in vocab_bert:
        emb[V[w]] = bert_emb[vocab_bert[w]]

np.save('dataset/data/bert_6871_256.npy',emb)
emb.shape

(5981, 256)


(6871, 256)

word2vec词向量

In [27]:
import numpy as np
import pickle as pkl

model = word2vec.Word2Vec.load("dataset/data/w2v/w2v_300_mincount5.model")
print(model)

V = pkl.load(open('dataset/data/vocab.pkl','rb'))
# for word in model.wv.vocab:
#     print(word)
emb = np.random.randn(len(V), 300) #* 0.000962582706903456
for w in V.keys():
    if w in model.wv.vocab:
        emb[V[w],:] = model.wv[w] * 100#正常的emb大概是1~2个小数点
#     break
emb.mean(), emb.std()

Word2Vec(vocab=6151, size=300, alpha=0.025)


(2.9378990724951244e-05, 0.09832080340287651)

In [117]:
np.save('dataset/data/w2v_300.npy',emb)
emb.shape

(6979, 300)

## 建模

In [ ]:
# import sys
# import os
# import importlib
# import time
# import numpy as np
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfTransformer  # 计算tfidf
# from sklearn.feature_extraction.text import CountVectorizer  # 计算df
# from sklearn.feature_extraction.text import TfidfVectorizer  # “一步到位”
# from gensim.models import word2vec
# from sklearn import preprocessing
# from sklearn import svm
# from sklearn.linear_model.logistic import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn import tree
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.externals import joblib
# from sklearn import metrics

# id_num = []
# word_article = []
# words_article = []
# label = []
# X_train = []
# y_train = []
# X_test = []
# y_test = []

# load_start = time.time()
# importlib.reload(sys)
# with open('E:/&数据集/DC竞赛数据集/train_set.csv', 'r', encoding='utf-8') as f:
#     lines = f.readlines()
#     for line in lines[1:20000]:
#         line = line.split(',')
#         X_train.append(line[2])
#         y_train.append(line[3])
#     for line in lines[20000:22000]:  # 划分出测试集
#         line = line.split(',')
#         X_test.append(line[2])
#         y_test.append(line[3])
# print("Time consumption on loading:", time.time() - load_start)

# train_start = time.time()
# w2v_size = 256
# w2v_model = word2vec.Word2Vec.load('./w2v/w2v.model')

# def tran_X(X_list):
#     X = [[] for i in range(len(X_list))]
#     for i in range(0, len(X_list)):
#         j = X_list[i].split(' ')  # 把str分成词list
#         X[i] += j
#     # print(X)
#     X_vectors = np.zeros((len(X), w2v_size))
#     i = 0
#     for words in X:
#         j = 0
#         for word in words:
#             if word in w2v_model.wv.vocab:
#                 X_vectors[i] += w2v_model[word]
#                 j += 1
#         X_vectors[i] /= j
#         i += 1
#     print("shape:", X_vectors.shape)

#     for i in range(len(X_vectors)):
#         for j in range(len(X_vectors[i])):
#             if np.isnan(X_vectors[i][j]):  # 去除NaN
#                 X_vectors[i][j] = 0
#     # X_vectors = preprocessing.scale(X_vectors) #标准化
#     X_vectors = preprocessing.MinMaxScaler().fit_transform(X_vectors)  # 归一化
#     # model_vectors = np.transpose(model_vectors) #转置
#     return X_vectors

X_train = tran_X(X_train)
print("Time consumption on training:", time.time() - train_start)
clf = svm.LinearSVC(C=1.2)
clf.fit(X_train, y_train)
# joblib.dump(clf, 'lineSVM_20000__.m')
print("Time consumption on training:", time.time() - train_start)

test_start = time.time()
# clf = joblib.load('lineSVM_20000__.m')
prediction = clf.predict(X_train)  # 训练集上准确率
print(metrics.classification_report(y_train, prediction))
X_test = tran_X(X_test)
prediction = clf.predict(X_test)  # 测试集上准确率
print(metrics.classification_report(y_test, prediction))
# print(metrics.confusion_matrix(y_test,prediction))
print("Time consumption on testing:", time.time() - test_start)


### 传统方法(TF-IDF+SVM)

In [15]:
# %%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range=(1,4), min_df=3, max_df=0.9, use_idf=1, smooth_idf=1, sublinear_tf=1)
vect.fit(x_train+x_dev+x_test) 
# x_train = vect.transform(x_train[:])
# X_test = vect.transform(X_test)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents=None, sublinear_tf=1,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=1,
        vocabulary=None)

In [16]:
import pickle
pickle.dump(vect, open("dataset/tfidf_vect.pickle", "wb"))
vect = pickle.load(open("dataset/tfidf_vect.pickle", "rb"))
vect.transform(x_train[:]).toarray().shape

(10, 19269679)

In [26]:
%%time
x_train_ = vect.transform(x_train[:])
x_dev_ = vect.transform(x_dev[:])

CPU times: user 3min 47s, sys: 23.9 s, total: 4min 11s
Wall time: 4min 14s


In [53]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

lsvc = svm.LinearSVC() #0.94516
sgdc = SGDClassifier(loss="modified_huber", alpha=0.000008, n_jobs=-1)
rf = RandomForestClassifier(n_jobs=-1) #0.71

clf = rf

k = None
clf.fit(x_train_[:k], y_train[:k])

pred = clf.predict(x_dev_)
dev_f1 = f1_score(pred, y_dev, average='macro')
print(dev_f1)

0.9451575162448084


In [30]:
with open('results/%s_result_%.4f.csv'%('TFIDF+ML', dev_f1), 'w') as f:
    f.write('label\n')
    for p in clf.predict(x_test_):
        f.write(str(p)+'\n')
        
prob = clf.predict_proba(x_dev_)
np.save('results/TFIDF+ML_prob', prob)

0.9420985068018918


In [ ]:
# from sklearn import svm
# from sklearn.linear_model.logistic import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier  
# from sklearn.neighbors import KNeighborsClassifier  
# from sklearn.linear_model import SGDClassifier
# from sklearn import tree
# from sklearn.externals import joblib
# from sklearn import metrics
 
# def training(train_set_in):
#     X = get_tfidf(train_set_in)
#     Y = []
#     for sente in cut_sentence(train_set_in):
#         Y.append(sente[0:1])
#     svc=svm.SVC(C=1,kernel='poly',degree=3,gamma=10,coef0=0) #选择模型 & 参数
#     lr = LogisticRegression()   
#     knn = KNeighborsClassifier()  
#     dt = tree.DecisionTreeClassifier()
    
#     rf = RandomForestClassifier()
# #     multi_target_forest = MultiOutputClassifier(rf)  # 构建多输出多分类器
# #     clf = multi_target_forest.fit(X, Y)
#     sgdc = SGDClassifier(loss="modified_huber", penalty="l2", alpha=0.00002, n_jobs=-1)# modified_huber; penalty="elasticnet"
# #     pre = clf.predict(X)
# #     print('多输出多分类器预测输出分类:\n',y_pred)
    
#     clf = sgdc.fit(X,Y) #训练模型
#     joblib.dump(clf,'train_model.m') #保存模型
    
# def testing(test_set_in):
#     file_original_txt = open(test_set_in,'r',encoding='utf-8')
#     original_txt = file_original_txt.readlines() #原始文本
#     i,j = 0,0
#     clf = joblib.load('train_model.m') #加载模型   
#     for sente in cut_sentence(test_set_in):
#         label = int(sente[0:1])
#         T = [] 
#         T.append(str(sente[2:]))      
#         tf=vectorizer.transform(T) #将数据进行转换，比如数据的归一化和标准化，将测试数据按照训练数据同样的模型进行转换，得到特征向量。
#         tfidf=transformer.transform(tf)
#         print(label,'\t',int(clf.predict(tfidf)),'\t',str(original_txt[j][2:])) 
#         if label == int(clf.predict(tfidf)):
#             i += 1 
#         j += 1  
#     print('\nprecision :',i/j)
    
# training('./project/dataset/train_set0.txt')
# testing('./project/dataset/test_set0.txt')


## 融合

In [ ]:
import os
import torch
import torch.nn.functional as F
from importlib import import_module
from utils import build_dataset, build_iterator, get_time_dif
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

dataset = 'dataset'  # 数据集THUCNews
embedding = 'random'  # 'embedding_SougouNews.npz'  'random'
use_word = True  # 中文用char更好，不会有OOV+学习char分布比word分布更容易(因为量多）

In [ ]:
%%time
def get_model(model_names):
    ret = []
    for model_name in model_names:
        x = import_module(name='models.' + model_name)  # 绝对导入
        config = x.Config(dataset, embedding)  # Config类
        config.train_path = dataset + '/data/dev.txt'
        vocab, train_data, dev_data, test_data = build_dataset(config, use_word)
        dev_iter = build_iterator(dev_data, config)
        test_iter = build_iterator(test_data, config)  # an object
        config.n_vocab = len(vocab)
        config.device = torch.device('cuda:0')
        model = x.Model(config)#.to(config.device)
        ret.append([model, config, dev_iter, test_iter])
    return ret

models = get_model(['TextRNN','TextCNN'])# TextCNN, TextRNN, FastText, TextRCNN, TextRNN_Att, DPCNN, Transformer

In [ ]:
models[0][0].load_state_dict(torch.load('dataset/saved_dict/TextRNN.ckpt_0.936358_59367.ckpt', map_location='cuda:0'))
models[1][0].load_state_dict(torch.load('dataset/saved_dict/TextCNN.ckpt_0.937892_74445.ckpt', map_location='cuda:0'))

In [ ]:
models

In [ ]:
def get_proba(models):
    ret = []
    # for model in models:
    m1, m2 = models
    model1, config, dev_iter, test_iter = m1
    model2, config, dev_iter, test_iter = m2

    model1.eval()
    model2.eval()

    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad():
        for texts, labels in dev_iter:
            outputs = model1(texts) + model2(texts)
            labels = labels.data.cpu().numpy()
            predic = torch.max(outputs.data, 1)[1].cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predic)

    f1 = f1_score(labels_all, predict_all, average='macro')
    print(f1)
    report = metrics.classification_report(labels_all, predict_all, target_names=config.class_list, digits=4)
    print("Precision, Recall and F1-Score...")
    print(report)
    ret.append([])
get_proba(models)
